In [1]:
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.211:7077") \
        .appName("Marina_Mota_Project")\
        .config("spark.dynamicAllocation.enabled", False)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores", 1)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

In [2]:
df = spark_session.read.json("hdfs://192.168.2.211:9000/user/ubuntu/input/RC_2005-12.bz2")

In [3]:
df_02 = spark_session.read.json("hdfs://192.168.2.211:9000/user/ubuntu/input/RC_2006-03.bz2")

In [17]:
df.show()

+----------+----------------------+-----------------+--------------------+----------------+-----------+-------------+------+------+---+--------+---------+------------+-----+--------+----------+------------+---+
|    author|author_flair_css_class|author_flair_text|                body|controversiality|created_utc|distinguished|edited|gilded| id| link_id|parent_id|retrieved_on|score|stickied| subreddit|subreddit_id|ups|
+----------+----------------------+-----------------+--------------------+----------------+-----------+-------------+------+------+---+--------+---------+------------+-----+--------+----------+------------+---+
|      frjo|                  null|             null|A look at Vietnam...|               0| 1134365188|         null| false|     0|c13|t3_17863| t3_17863|  1473738411|    2|   false|reddit.com|        t5_6|  2|
|   zse7zse|                  null|             null|The site states "...|               0| 1134365725|         null| false|     0|c14|t3_17866| t3_17866|  

In [13]:
df.printSchema()

root
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: long (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- edited: boolean (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- retrieved_on: long (nullable = true)
 |-- score: long (nullable = true)
 |-- stickied: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- ups: long (nullable = true)



In [4]:
df2 = df.groupBy('author').agg({'score': 'mean'}).orderBy('avg(score)', ascending=False).withColumnRenamed('avg(score)', 'mean_score')

In [10]:
df2.show()

+--------------+-----------------+
|        author|       mean_score|
+--------------+-----------------+
|TenebraeVision|             16.0|
|          Grue|             16.0|
|        fergie|             12.0|
|        bstard|             11.0|
|         sempf|9.666666666666666|
|     bolinfest|              7.0|
|      kanagawa|              7.0|
|    HiggsBoson|              7.0|
|    paulgraham|6.533333333333333|
|         arkas|              6.5|
|      dstowell|6.333333333333333|
|        kelyse|              6.0|
|     FutureAEI|              6.0|
|       gernika|              6.0|
|          chao|              6.0|
|       spolsky|              6.0|
|       AaronSw|              6.0|
|      bwringer|              6.0|
|        willfe|              6.0|
|          bosk|              5.5|
+--------------+-----------------+
only showing top 20 rows



In [9]:
df.count()

1075

In [4]:
reddit_rdd = df.select("body").rdd.flatMap(lambda x: x)
reddit_rdd.take(10)

['A look at Vietnam and Mexico exposes the myth of market liberalisation.',
 'The site states "What can I use it for? Meeting notes, Reports, technical specs Sign-up sheets, proposals and much more...", just like any other new breeed of sites that want us to store everything we have on the web. And they even guarantee multiple levels of security and encryption etc. But what prevents these web site operators fom accessing and/or stealing Meeting notes, Reports, technical specs Sign-up sheets, proposals and much more, for competitive or personal gains...? I am pretty sure that most of them are honest, but what\'s there to prevent me from setting up a good useful site and stealing all your data? Call me paranoid - I am.',
 'Jython related topics by Frank Wierzbicki',
 '[deleted]',
 'Saft is by far the best extension you could tak onto your Safari',
 '[deleted]',
 'How to take panoramic shots simply',
 'I donâ\x80\x99t know where they came up with this stuff, but Qube Web Search Client has

In [5]:
reddit_rdd_02 = df_02.select("body").rdd.flatMap(lambda x: x)
reddit_rdd_02.take(10)

["It's not garbage, its fear mongering.",
 "Actually, it's nice to have a new idea too, especially when you say stuff about how your thing is gonna do for the web what the Mac did for the personal computer. If there was any evidence of that kind of thing to be smelled anywhere, I think you'd find a little more willingness to wait and see on the execution.",
 "I've read them, and they are not that funny. I would suggest you go to bash.org and read top 100, those are much better.",
 "If you look at a lot of these changes, they are really extracted refinements rather than bloating additions. They aim at making things more DRY and eliminating workarounds that used to be more clumsy. So while the Rails API may gain a few methods, it will actually make the programmer's experience using Rails more consistent and streamlined.\r\n\r\nThe beauty of a framework like rails is that you can use it without leveraging its full api - go ahead and write your stuff 'old skool' in 15 lines of code, and a 

In [6]:
reddit_rdd2 = reddit_rdd.filter(lambda row: row != "[deleted]")
reddit_rdd2.take(10)

['A look at Vietnam and Mexico exposes the myth of market liberalisation.',
 'The site states "What can I use it for? Meeting notes, Reports, technical specs Sign-up sheets, proposals and much more...", just like any other new breeed of sites that want us to store everything we have on the web. And they even guarantee multiple levels of security and encryption etc. But what prevents these web site operators fom accessing and/or stealing Meeting notes, Reports, technical specs Sign-up sheets, proposals and much more, for competitive or personal gains...? I am pretty sure that most of them are honest, but what\'s there to prevent me from setting up a good useful site and stealing all your data? Call me paranoid - I am.',
 'Jython related topics by Frank Wierzbicki',
 'Saft is by far the best extension you could tak onto your Safari',
 'How to take panoramic shots simply',
 'I donâ\x80\x99t know where they came up with this stuff, but Qube Web Search Client has taken the market by surpris

In [7]:
reddit_rdd_022 = reddit_rdd_02.filter(lambda row: row != "[deleted]")
reddit_rdd_022.take(10)

["It's not garbage, its fear mongering.",
 "Actually, it's nice to have a new idea too, especially when you say stuff about how your thing is gonna do for the web what the Mac did for the personal computer. If there was any evidence of that kind of thing to be smelled anywhere, I think you'd find a little more willingness to wait and see on the execution.",
 "I've read them, and they are not that funny. I would suggest you go to bash.org and read top 100, those are much better.",
 "If you look at a lot of these changes, they are really extracted refinements rather than bloating additions. They aim at making things more DRY and eliminating workarounds that used to be more clumsy. So while the Rails API may gain a few methods, it will actually make the programmer's experience using Rails more consistent and streamlined.\r\n\r\nThe beauty of a framework like rails is that you can use it without leveraging its full api - go ahead and write your stuff 'old skool' in 15 lines of code, and a 

In [8]:
import re
def lower_rdd(lines):
      lines = lines.lower()
      words = re.findall(r'[a-zA-Z]+', lines)
      return words
reddit_words = reddit_rdd2.map(lower_rdd)
reddit_words.take(10)

[['a',
  'look',
  'at',
  'vietnam',
  'and',
  'mexico',
  'exposes',
  'the',
  'myth',
  'of',
  'market',
  'liberalisation'],
 ['the',
  'site',
  'states',
  'what',
  'can',
  'i',
  'use',
  'it',
  'for',
  'meeting',
  'notes',
  'reports',
  'technical',
  'specs',
  'sign',
  'up',
  'sheets',
  'proposals',
  'and',
  'much',
  'more',
  'just',
  'like',
  'any',
  'other',
  'new',
  'breeed',
  'of',
  'sites',
  'that',
  'want',
  'us',
  'to',
  'store',
  'everything',
  'we',
  'have',
  'on',
  'the',
  'web',
  'and',
  'they',
  'even',
  'guarantee',
  'multiple',
  'levels',
  'of',
  'security',
  'and',
  'encryption',
  'etc',
  'but',
  'what',
  'prevents',
  'these',
  'web',
  'site',
  'operators',
  'fom',
  'accessing',
  'and',
  'or',
  'stealing',
  'meeting',
  'notes',
  'reports',
  'technical',
  'specs',
  'sign',
  'up',
  'sheets',
  'proposals',
  'and',
  'much',
  'more',
  'for',
  'competitive',
  'or',
  'personal',
  'gains',
  'i',

In [9]:
reddit_words_02 = reddit_rdd_022.map(lower_rdd)
reddit_words_02.take(10)

[['it', 's', 'not', 'garbage', 'its', 'fear', 'mongering'],
 ['actually',
  'it',
  's',
  'nice',
  'to',
  'have',
  'a',
  'new',
  'idea',
  'too',
  'especially',
  'when',
  'you',
  'say',
  'stuff',
  'about',
  'how',
  'your',
  'thing',
  'is',
  'gonna',
  'do',
  'for',
  'the',
  'web',
  'what',
  'the',
  'mac',
  'did',
  'for',
  'the',
  'personal',
  'computer',
  'if',
  'there',
  'was',
  'any',
  'evidence',
  'of',
  'that',
  'kind',
  'of',
  'thing',
  'to',
  'be',
  'smelled',
  'anywhere',
  'i',
  'think',
  'you',
  'd',
  'find',
  'a',
  'little',
  'more',
  'willingness',
  'to',
  'wait',
  'and',
  'see',
  'on',
  'the',
  'execution'],
 ['i',
  've',
  'read',
  'them',
  'and',
  'they',
  'are',
  'not',
  'that',
  'funny',
  'i',
  'would',
  'suggest',
  'you',
  'go',
  'to',
  'bash',
  'org',
  'and',
  'read',
  'top',
  'those',
  'are',
  'much',
  'better'],
 ['if',
  'you',
  'look',
  'at',
  'a',
  'lot',
  'of',
  'these',
  'cha

In [19]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all-nltk


       | 
       | Downloading package abc to /home/ubuntu/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /home/ubuntu/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package biocreative_ppi to
       |     /home/ubuntu/nltk_data...
       |   Package biocreative_ppi is already up-to-date!
       | Downloading package brown to /home/ubuntu/nltk_data...
       |   Package brown is already up-to-date!
       | Downloading package brown_tei to /home/ubuntu/nltk_data...
       |   Package brown_tei is already up-to-date!
       | Downloading package cess_cat to /home/ubuntu/nltk_data...
       |   Package cess_cat is already up-to-date!
       | Downloading package cess_esp to /home/ubuntu/nltk_data...
       |   Package cess_esp is already up-to-date!
       | Downloading package chat80 to /home/ubuntu/nltk_data...
       |   Package chat80 is already up-to-date!
       | Downloading package city_databas

       |   Package timit is already up-to-date!
       | Downloading package toolbox to /home/ubuntu/nltk_data...
       |   Package toolbox is already up-to-date!
       | Downloading package treebank to /home/ubuntu/nltk_data...
       |   Package treebank is already up-to-date!
       | Downloading package twitter_samples to
       |     /home/ubuntu/nltk_data...
       |   Package twitter_samples is already up-to-date!
       | Downloading package udhr to /home/ubuntu/nltk_data...
       |   Package udhr is already up-to-date!
       | Downloading package udhr2 to /home/ubuntu/nltk_data...
       |   Package udhr2 is already up-to-date!
       | Downloading package unicode_samples to
       |     /home/ubuntu/nltk_data...
       |   Package unicode_samples is already up-to-date!
       | Downloading package universal_treebanks_v20 to
       |     /home/ubuntu/nltk_data...
       |   Package universal_treebanks_v20 is already up-to-date!
       | Downloading package verbnet to /home


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [10]:
from nltk.corpus import stopwords
stopword_list = stopwords.words('english')
stopword_list.append('http')
stopword_list.append('com')

In [11]:
def removeStopWordsFunct(x):
    filteredSentence = [w for w in x if not w in stopword_list]
    return filteredSentence
stopword_reddit = reddit_words.map(removeStopWordsFunct)

In [12]:
stopword_reddit.take(10)

[['look', 'vietnam', 'mexico', 'exposes', 'myth', 'market', 'liberalisation'],
 ['site',
  'states',
  'use',
  'meeting',
  'notes',
  'reports',
  'technical',
  'specs',
  'sign',
  'sheets',
  'proposals',
  'much',
  'like',
  'new',
  'breeed',
  'sites',
  'want',
  'us',
  'store',
  'everything',
  'web',
  'even',
  'guarantee',
  'multiple',
  'levels',
  'security',
  'encryption',
  'etc',
  'prevents',
  'web',
  'site',
  'operators',
  'fom',
  'accessing',
  'stealing',
  'meeting',
  'notes',
  'reports',
  'technical',
  'specs',
  'sign',
  'sheets',
  'proposals',
  'much',
  'competitive',
  'personal',
  'gains',
  'pretty',
  'sure',
  'honest',
  'prevent',
  'setting',
  'good',
  'useful',
  'site',
  'stealing',
  'data',
  'call',
  'paranoid'],
 ['jython', 'related', 'topics', 'frank', 'wierzbicki'],
 ['saft', 'far', 'best', 'extension', 'could', 'tak', 'onto', 'safari'],
 ['take', 'panoramic', 'shots', 'simply'],
 ['know',
  'came',
  'stuff',
  'qube',
 

In [13]:
stopword_reddit_02 = reddit_words_02.map(removeStopWordsFunct)
stopword_reddit_02.take(10)

[['garbage', 'fear', 'mongering'],
 ['actually',
  'nice',
  'new',
  'idea',
  'especially',
  'say',
  'stuff',
  'thing',
  'gonna',
  'web',
  'mac',
  'personal',
  'computer',
  'evidence',
  'kind',
  'thing',
  'smelled',
  'anywhere',
  'think',
  'find',
  'little',
  'willingness',
  'wait',
  'see',
  'execution'],
 ['read',
  'funny',
  'would',
  'suggest',
  'go',
  'bash',
  'org',
  'read',
  'top',
  'much',
  'better'],
 ['look',
  'lot',
  'changes',
  'really',
  'extracted',
  'refinements',
  'rather',
  'bloating',
  'additions',
  'aim',
  'making',
  'things',
  'dry',
  'eliminating',
  'workarounds',
  'used',
  'clumsy',
  'rails',
  'api',
  'may',
  'gain',
  'methods',
  'actually',
  'make',
  'programmer',
  'experience',
  'using',
  'rails',
  'consistent',
  'streamlined',
  'beauty',
  'framework',
  'like',
  'rails',
  'use',
  'without',
  'leveraging',
  'full',
  'api',
  'go',
  'ahead',
  'write',
  'stuff',
  'old',
  'skool',
  'lines',
  

In [14]:
def count_words(lines):
    counts = lines.flatMap(lambda line: line) \
        .map(lambda word: (word, 1)) \
        .reduceByKey(lambda a, b: a + b) 
    return counts.sortBy(lambda a: a[1], False)

In [30]:
%%time
word_counts = count_words(stopword_reddit)
word_counts.collect()

CPU times: user 24.5 ms, sys: 3.04 ms, total: 27.6 ms
Wall time: 2.1 s


In [27]:
%%time
word_counts_02 = count_words(stopword_reddit_02)
word_counts_02.collect()

CPU times: user 25.7 ms, sys: 3.37 ms, total: 29 ms
Wall time: 2.31 s


In [ ]:
# gives a single float value
psutil.cpu_percent()
# gives an object with many fields
psutil.virtual_memory()
# you can convert that object to a dictionary 
dict(psutil.virtual_memory()._asdict())

In [7]:
# release the cores for another application!
spark_context.stop()